In [3]:
import pandas as pd
from pathlib import Path
from typing import List, Union

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

directory = "../SRB"
mp_path = Path(directory) / Path("Serbia_MPs_final_20221007.xlsx")
parties_path = Path(directory) / Path("Serbia_parties_final_20221007.xlsx")

In [23]:
mpdf = pd.read_excel(str(mp_path))#.dropna()
partiesdf = pd.read_excel(str(parties_path))

## Check and impute missing values:

In [24]:
partiesnas = partiesdf.isna().any(axis="columns")
# Only hit: SPDO, term 7.
# According to Wikipedia: Coalition of Democratic Party of Serbia – New Serbia – dr Vojislav Koštunica (DSS, NS, JS, SDPO)
# It doesn't say if SDPO got in coalition, but I can check the others.
# c1 = partiesdf.term2.astype(int) == 7
# c2 = partiesdf.party.isin(["DSS", "NS", "JS", "SPDO"])
# partiesdf[c1&c2]
# all others have ruling=1, so I'm imputing ruling=1 also for SPDO

partiesdf.loc[57, "ruling"] = 1
partiesdf[partiesnas]

,codeparty,term1,term2,party,full_name,established,chairman,ideology_LR,party_family,election_result,no_seats,coalition,coalition_composition,ruling
57,P58,2007-2008,7,SDPO,Srpski demokratski pokret obnove,2005,Vojislava Mihailovića/Veroljuba Stevanovića,4,6;7,16.55,2,1,JS-DSS-NS,1.0


In [15]:
mpnas = mpdf.isna().any(axis="columns")
# Nadije Bećiri: Poslanička grupa Ujedinjena dolina-SDA Sandžaka
# This has been identified as party SDA
# Assign:
mpdf.loc[2194, "party"] = "SDA"


# Vladan Glišić:  Poslanički klub: Narodni poslanici koji nisu članovi poslaničkih grupa 
# Can't impute this. Leaving NaN for now.

# Add birth year to row 113:
mpdf.loc[113, "date_of_birth"] = mpdf.loc[113, "year_of_birth"]

mpdf[mpnas]

,codemp,order_id,term1,term2,term_id,type_of_list,fullname,firstname,lastname,party,date_of_birth,year_of_birth,gender,place_of_birth,field_of_study,education_y,bp_lat,bp_lon
113,M179,114,1997-2000,4,114,various,"Kovačević, Dejan",Dejan,Kovačević,LK,1937,1937,0,Beograd,2,16,44.688045,20.22171
2194,M1315,2195,2020-2022,12,17,normal,"Bećiri, Nadije",Nadije,Bećiri,NaN,1961,1961,1,"Bujanovac, Lučane",4,16,42.448444,21.705894
2226,M1331,2227,2020-2022,12,49,normal,"Glišić, Vladan",Vladan,Glišić,NaN,1970,1970,0,Priština,5,16,42.629566,21.108718


## Checking if the names are correctly segmented:

They are, no such pathologies as with 

I found entry

```
1957	Jovanović	Jovanović1, Nataša
```
Upon further inspection I also found the following pathological entries:

```
M103    Kovačević, Borisav        #-> Exists: https://otvoreniparlament.rs/poslanik/8455
        Kovačević, Borislav       #-> Only 2 hits in metadata. No hits on otvoreni parlament. Probably misspelling.
M1141   Nikolić Vukajlović, Vesna #-> Obviously wrong. No hits in the metadata
        Nikolić-Vukajlović, Vesna
M67     Hasanović Korać, Biljana  #-> Obviously wrong. No hits in the metadata
        Hasanović-Korać, Biljana
M781    Jovanović, Nataša         #-> All three versions have hits in metadata. Likely the same person (Only one hit on otvoreni parlament)
        Jovanović1, Nataša        #-> All three versions have hits in metadata. Likely the same person (Only one hit on otvoreni parlament)
M782    Jovanović2, Nataša        #-> All three versions have hits in metadata. Likely the same person (Only one hit on otvoreni parlament)
```

This will be corrected in the first three cases. 
